In [ ]:
import wandb
wand_key = '1c0992a8cc7785f33c39e7f1b62841cf07d7f19e'
wandb.login(key = wand_key)

In [ ]:
import os
import numpy as np
import torch

# Addresses

class Address:
    def __init__(self):
        '''
        Stores all the addresses used in project
        '''
        # Inputs
        self.data = "../input/mammography"
        self.processed_data = "data.pkl"

        # Coco
        self.coco = os.path.join(self.data, 'coco_1k')
        self.coco_annot = os.path.join(self.coco, 'annotations')
        self.coco_annot_train = os.path.join(self.coco_annot, 'instances_train2017.json')
        self.coco_annot_val = os.path.join(self.coco_annot, 'instances_val2017.json')
        self.coco_img_train = os.path.join(self.coco, 'train2017')
        self.coco_img_val = os.path.join(self.coco, 'val2017')

        # Test
        self.test = os.path.join(self.data, 'test')
        self.test_img = os.path.join(self.test, 'images')
        self.test_label = os.path.join(self.test, 'labels')
        self.predictions = os.path.join(self.test, 'predictions')

        # Yolo
        self.yolo = os.path.join(self.data, 'yolo_1k')
        self.yolo_train = os.path.join(self.yolo, 'train')
        self.yolo_train_img = os.path.join(self.yolo_train, 'images')
        self.yolo_train_label = os.path.join(self.yolo_train, 'labels')
        self.yolo_val = os.path.join(self.yolo, 'val')
        self.yolo_val_img = os.path.join(self.yolo_val, 'images')
        self.yolo_val_labels = os.path.join(self.yolo_val, 'labels')

        # Models
        self.result = "results/"
        self.model_frcnn = os.path.join(self.result, 'frcnn')
        self.model_def_detr = os.path.join(self.result, 'deformable_dtr')

        # Temp
        self.temp = "temp/"

    def create_dir(self, dir_list = None):
        '''
        Function to create directories in dir_list. If dir_list is None then create all directories of address.
        '''
        if dir_list == None:
            dir_list = [self.temp, self.result, self.model_frcnn, self.model_def_detr]
        for address in dir_list:
            if not os.path.exists(address):
                os.mkdir(address)

    def _delete_folder_content(self, folder_addr):
        '''
        Deletes all the content of folder_addr
        '''
        if os.path.exists(folder_addr):
            for file in os.listdir(folder_addr):
                address = os.path.join(folder_addr, file)
                if os.path.isdir(address):
                    self._delete_folder_content(address)
                    os.removedirs(address)
                else:
                    os.remove(address)

    def clean(self, file_list = None):
        '''
        Deletes all the content in file_list
        '''
        if file_list == None:
            file_list = [self.temp]
        for address in file_list:
            self._delete_folder_content(address)

addr = Address()
addr.clean()
# addr.clean([addr.model_def_detr])
addr.create_dir()

class HyperParameters:
    def __init__(self):
        '''
        Stores all Hyperparameters used for training of model
        '''
        # Training
        self.batch_size = 16
        self.num_epoch = 100
        self.grad_clip = 1.0

        # Data
        self.num_train = 2240
        self.num_val = 218
        self.train_step = self.num_epoch*(self.num_train//self.batch_size)
        self.resolution = (256, 512)
        
        # Learning Rate
        self.lr = 1e-5
        self.warmup_step = self.train_step//40
        self.decay_step = self.train_step//2
        self.decay_rate = 0.5

    def lr_schedule(self, step):
        '''
        Getting learning rate as function of train steps completed (Exponential decay with linear warmup)
        '''
        if step <= self.warmup_step:
            return step/self.warmup_step
        else:
            return self.decay_rate**((step-self.warmup_step)/self.decay_step)

    def create_report(self, addr):
        with open(os.path.join(addr, 'param.txt'), 'w') as file:
            file.writelines([
                f'Training:',
                f'\n\tBatch Size:       {self.batch_size}',
                f'\n\tNum Epoch:        {self.num_epoch}',
                f'\n\tGrad Clip:        {self.grad_clip}',
                f'\n\nData:',  
                f'\n\tNum Train:        {self.num_train}',
                f'\n\tNum Val:          {self.num_val}',
                f'\n\tTrain Step:       {self.train_step}',
                f'\n\tResolution:       {self.resolution}',
                f'\n\nLearning Rate:',  
                f'\n\tlr:               {self.lr}',
                f'\n\tWarmup Step:      {self.warmup_step}',
                f'\n\tDecay Step:       {self.decay_step}',
                f'\n\tDecay Rate:       {self.decay_rate}',
            ])

param = HyperParameters()

# Random Seed and CUDA

random_seed = 68
device = "cpu"
torch.manual_seed(random_seed)
np.random.seed(random_seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(random_seed)
    device = "cuda"
print(f"Working with device {device}")

In [ ]:
import json
import torchvision
import transformers

# Dataset

# model_checkpoint = "SenseTime/deformable-detr"
model_checkpoint = "SenseTime/deformable-detr-single-scale"
# model_checkpoint = "facebook/deformable-detr-box-supervised"

class DataSet(torch.utils.data.Dataset):
    def __init__(self, address_img, address_annot):
        '''
        Creates Dataset of images on given address.
        '''
        self.address_img = address_img
        self.temp_annotation = None
        with open(address_annot, 'rb') as file:
            self.annotation = json.load(file)
        self.label2id = {cat['name']: cat['id'] for cat in self.annotation['categories']}
        self.id2label = {cat['id']: cat['name'] for cat in self.annotation['categories']}
        self.preprocess = torchvision.transforms.Compose([torchvision.transforms.GaussianBlur(5),
                                                          torchvision.transforms.ColorJitter(contrast=50),
                                                          torchvision.transforms.Grayscale()])
        self.beautify()
        self.feature_extractor = transformers.DeformableDetrFeatureExtractor.from_pretrained(model_checkpoint)
        self.feature_extractor.size = {'shortest_edge': param.resolution[0], 'longest_edge': param.resolution[1]}

    def __len__(self):
        return len(self.annotation)
    
    def __getitem__(self, idx):
        if 'image' not in self.annotation[idx]:
            img_addr = os.path.join(self.address_img, self.annotation[idx]['file_name'])
            img = torchvision.datasets.folder.default_loader(img_addr)
            self.annotation[idx]['image'] = img
        return self.transform(self.annotation[idx])
    
    def beautify(self):
        img_info = self.annotation['images']
        annotations = self.annotation['annotations']
        id_dict = {}
        for img in img_info:
            img['objects'] = []
            id_dict[img['id']] = img
        for annot in annotations:
            img = id_dict[annot['image_id']]
            img['objects'].append(annot)
        self.annotation = sorted(id_dict.values(), key=lambda item: item['id'])
        self.file_dict = {elem['file_name']: elem for elem in self.annotation}

    def transform(self, elem):
        # images = self.preprocess(elem["image"]).unsqueeze(0)
        images = elem["image"]
        ids_ = elem["id"]
        objects = elem["objects"]
        targets = {"image_id": ids_, "annotations": objects}
        feature = self.feature_extractor(images=images, annotations=targets, return_tensors="pt")
        feature['file_name'] = elem['file_name']
        return feature

    def collate(self, batch):
        pixel_values = [item["pixel_values"].squeeze(0) for item in batch]
        encoding = self.feature_extractor.pad(
            pixel_values, return_tensors="pt"
        )
        labels = [item["labels"][0] for item in batch]
        batch = {} # collated batch  
        batch['pixel_values'] = encoding['pixel_values']
        batch["pixel_mask"] = encoding["pixel_mask"]
        batch["labels"] = labels
        return batch

dataset_train = DataSet(addr.coco_img_train, addr.coco_annot_train)
dataset_val = DataSet(addr.coco_img_val, addr.coco_annot_val)

In [ ]:
# Model
config = transformers.DeformableDetrConfig.from_pretrained(
    model_checkpoint, 
    num_labels=1, 
    id2label=dataset_train.id2label,
    label2id=dataset_train.label2id,
)

model = transformers.DeformableDetrForObjectDetection(config)

for p in model.parameters():
    p.requires_grad = True


In [ ]:
# Training
training_args = transformers.TrainingArguments(
    output_dir = addr.model_def_detr,
    num_train_epochs = param.num_epoch,
    per_device_train_batch_size = param.batch_size,
    learning_rate = param.lr,
    warmup_steps = param.warmup_step,
    max_grad_norm=param.grad_clip,
    evaluation_strategy = 'epoch',
    save_strategy = 'epoch',
    save_total_limit = 2,
    logging_strategy = 'epoch',
    use_cpu = False,
    load_best_model_at_end = True
)

trainer = transformers.Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset_train,
    eval_dataset=dataset_val,
    tokenizer=dataset_train.feature_extractor,
    data_collator=dataset_train.collate,
)

print("Started Training")

# trainer.train()


In [ ]:
import PIL.Image
import matplotlib.pyplot as plt
import PIL

# Evaluation

iou_threshold = [0, 0.1, 0.25, 0.5, 1]
nms_addr = os.path.join(addr.model_def_detr, 'nms')
addr.create_dir([nms_addr])

def plot_results(pil_img, boxes, orig_boxes, file_addr):
    plt.figure(figsize=(5, 10))
    plt.imshow(pil_img)
    ax = plt.gca()
    for (xmin, ymin, xmax, ymax) in boxes.tolist():
        ax.add_patch(plt.Rectangle((xmin, ymin), xmax - xmin, ymax - ymin,
                                   fill=False, color='r', linewidth=1))
    for (xcen, ycen, xwidth, ywidth) in orig_boxes:
        ax.add_patch(plt.Rectangle((xcen-xwidth/2, ycen-ywidth/2), xwidth, ywidth,
                                   fill=False, color='b', linewidth=1))
    plt.axis('off')
    plt.savefig(file_addr)
    plt.close()

final_model = os.path.join(addr.model_def_detr, f'checkpoint')
model = transformers.DeformableDetrForObjectDetection.from_pretrained(final_model).to(device)
model.eval()

with torch.no_grad():
    for data in dataset_train:
        # Loading data
        filename = data['file_name']
        del data['file_name']

        # Getting original box
        orig_data = dataset_train.file_dict[filename]
        orig_boxes = [elem['bbox'] for elem in orig_data['objects']]

        # Passing through the model
        features = dataset_train.feature_extractor(images=orig_data['image'])
        features['pixel_values'] = torch.tensor(np.stack(features['pixel_values'])).to(device)
        features['pixel_mask'] = torch.tensor(np.stack(features['pixel_mask'])).to(device)
        output = model(**features)
        img = PIL.Image.open(os.path.join(addr.coco_img_train, filename))
        target_sizes = torch.tensor([img.size[::-1]])

        # Plotting with nms
        results = dataset_train.feature_extractor.post_process_object_detection(output, target_sizes=target_sizes, threshold=0.0)[0]
        results['boxes'] = results['boxes'].cpu()
        results['scores'] = results['scores'].cpu()
        for i in range(len(iou_threshold)):
            ind = torchvision.ops.nms(results['boxes'], results['scores'], iou_threshold[i])
            address = os.path.join(nms_addr, filename[:-4])
            addr.create_dir([address])
            plot_results(img, results['boxes'][ind], orig_boxes, os.path.join(address, f"{iou_threshold[i]}.png"))